In [3]:
from pathlib import Path
import glob, pandas as pd

# Try a few likely locations/names
candidates = [
    Path("GAMING 2.csv"),
    Path("GAMING_2.csv"),
    Path("data") / "GAMING 2.csv",
    Path("inputs") / "GAMING 2.csv",
    Path.cwd() / "GAMING 2.csv",
    Path.home() / "Downloads" / "GAMING 2.csv",
]

found = next((p for p in candidates if p.exists()), None)
if not found:
    matches = glob.glob("**/GAMING*2*.csv", recursive=True)
    if matches:
        found = Path(matches[0])
    else:
        raise FileNotFoundError(
            "Couldn't find 'GAMING 2.csv'. Move it next to the notebook OR set CSV_PATH to the full path."
        )

CSV_PATH = str(found)
df = pd.read_csv(CSV_PATH)
print("Loaded:", CSV_PATH, "→", df.shape)
print("Columns:", list(df.columns))


Loaded: GAMING 2.csv → (242, 19)
Columns: ['ID', 'ClientID', 'Status', 'Title', 'Field', 'Service', 'Category', 'CreatedAt', 'UpdatedAt', 'AcknowledgedAt', 'ProcessingAt', 'ClosedAt', 'ReopenedAt', 'LastSimplyReplyAt', 'LastClientReplyAt', 'ClientName', 'Time to Close', 'TAM ', 'Closer']


In [4]:
df

,ID,ClientID,Status,Title,Field,Service,Category,CreatedAt,UpdatedAt,AcknowledgedAt,ProcessingAt,ClosedAt,ReopenedAt,LastSimplyReplyAt,LastClientReplyAt,ClientName,Time to Close,TAM,Closer
0,11531,4172.0,closed,[**Freeview**] EPG schedule gap,NaN,schedules,missing_data,45903.00704,45903.65321,45903.02801,45903.10142,45903.65321,NaN,45903.65322,45903.00704,Freeview / Freely UK,1.550806e+01,Aneet,Annet
1,11532,1638.0,closed,Missing detailed information for program id 14...,data_points_id,editorial,missing_data,45903.22057,45905.68690,45903.22057,45903.44718,45905.68690,NaN,45905.68719,45903.60337,Telia / TV 4 Media AB,5.919194e+01,Theo,Theo
2,11537,3829.0,closed,weapons - Baltics,technical_data_parental_ratings,editorial,incorrect_data,45903.28135,45903.28307,45903.28135,NaN,45903.28294,NaN,45903.28332,45903.28135,SF Anytime,3.805556e-02,Theo,Zarmeen
3,11538,87.0,closed,DNA TV / EPG shows episode's name instead of s...,data_points_title,ingest,incorrect_data,45903.30549,45903.71740,45903.30549,45903.44925,45903.71740,NaN,45903.71743,45903.30549,DNA Oy,9.885833e+00,Theo,Theo
4,11539,11.0,closed,EPG: SVT1 HD / / There is no title on this pr...,NaN,delivery,missing_data,45903.30564,45904.31837,45903.30564,45903.66899,45904.31837,NaN,45904.31839,45903.30650,Stofa / Norlys,2.430556e+01,Theo,Theo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,11781,3851.0,processing,F√∫tbol de Argentina - LPF Torneo Clausura 202...,data_points_title,editorial,change_request,45909.74948,45909.75968,45909.74948,45909.75950,NaN,NaN,45909.83729,45909.74948,Wiltel Argentina,-1.101834e+06,Gui,NaN
238,11782,3851.0,processing,A Lot Like Love on HBO POP Caribbean 09.09.202...,data_points_title,editorial,incorrect_data,45909.76206,45909.77005,45909.76531,45909.76993,NaN,NaN,NaN,45909.76206,Wiltel Argentina,-1.101834e+06,Gui,NaN
239,11783,4157.0,processing,Wrong EPG schedule ‚Äì TNTS 4 ‚Äì EPG schedule...,delivery_feed,schedules,missing_data,45909.76745,45909.76897,45909.76745,45909.76880,NaN,NaN,45909.79808,45909.76745,BT (Youview delivery),-1.101834e+06,Aneet,NaN
240,11784,1764.0,processing,Innspire - SimplyTV IDs check-up,delivery_service_addition,delivery,change_request,45909.77439,45909.77522,45909.77439,45909.77495,NaN,NaN,45909.77755,45909.77786,Innspire,-1.101835e+06,Sergiu,NaN


In [6]:
# ---- Lock column mappings (from your header screenshot) ----
closer_col   = "Closer"
client_col   = "ClientName"
created_col  = "CreatedAt"
closed_col   = "ClosedAt"
reopen_col   = "ReopenedAt"
ttc_col      = "Time to Close"   # note the space
tam_col      = "TAM"             # only if you still want to reference TAM

# Clean column names (just in case there are leading/trailing spaces)
df.columns = [c.strip() for c in df.columns]

# Parse timestamps (dayfirst=True if your dates are dd/mm/yy)
for c in [created_col, closed_col, reopen_col, "UpdatedAt",
          "AcknowledgedAt", "ProcessingAt", "LastSimplyReplyAt", "LastClientReplyAt"]:
    if c in df.columns:
        df[c] = pd.to_datetime(df[c], errors="coerce", dayfirst=True)

# Keep only rows with a valid closer and a closed time
df[closer_col] = df[closer_col].astype(str).str.strip()
df = df[~df[closer_col].str.lower().isin(["not found","","none","nan"])].copy()
df = df.dropna(subset=[closed_col])

# Ensure the time-to-close column exists; compute if missing
if ttc_col not in df.columns:
    df[ttc_col] = (df[closed_col] - df[created_col]).dt.total_seconds()/3600.0

# Re-open flag
df["_reopened_flag"] = df[reopen_col].notna() if reopen_col in df.columns else False

# Quick sanity peek
df[[closer_col, client_col, created_col, closed_col, reopen_col, ttc_col]].head()



,Closer,ClientName,CreatedAt,ClosedAt,ReopenedAt,Time to Close
0,Annet,Freeview / Freely UK,1970-01-01 00:00:00.000045903,1970-01-01 00:00:00.000045903,NaT,15.508056
1,Theo,Telia / TV 4 Media AB,1970-01-01 00:00:00.000045903,1970-01-01 00:00:00.000045905,NaT,59.191944
2,Zarmeen,SF Anytime,1970-01-01 00:00:00.000045903,1970-01-01 00:00:00.000045903,NaT,0.038056
3,Theo,DNA Oy,1970-01-01 00:00:00.000045903,1970-01-01 00:00:00.000045903,NaT,9.885833
4,Theo,Stofa / Norlys,1970-01-01 00:00:00.000045903,1970-01-01 00:00:00.000045904,NaT,24.305556


In [8]:
# 4.1 Clients with the most tickets (new CSV uses 'ClientName')
import plotly.express as px

client_col = "ClientName"  # from your new file header

clients_df = (
    df.assign(**{client_col: df[client_col].astype(str).str.strip()})
      .loc[df[client_col].notna() & (df[client_col].astype(str).str.strip() != "")]
      .groupby(client_col, as_index=False)
      .size()
      .rename(columns={"size": "tickets_count"})
      .sort_values("tickets_count", ascending=False)
)

fig = px.bar(
    clients_df.head(15),
    x="tickets_count", y=client_col, orientation="h", color="tickets_count",
    title="👑 Which client generated the most tickets?",
    labels={"tickets_count": "Tickets", client_col: "Client"},
    template="simple_white"
)
fig.update_yaxes(categoryorder="total ascending")
fig.update_layout(showlegend=False)
fig.show()


In [11]:
# Who received the most tickets? (by TAM)
import plotly.express as px

tam_col = "TAM"  # column name from your CSV

tam_recv_df = (
    df
    # clean up the TAM column
    .assign(**{tam_col: df[tam_col].astype("string").str.strip().str.replace(r"\s+", " ", regex=True)})
    # drop blanks / NaNs / "not found"
    .loc[lambda d: d[tam_col].notna()
                  & (d[tam_col] != "")
                  & (~d[tam_col].str.lower().isin(["not found", "nan", "none"])) ]
    # aggregate
    .groupby(tam_col, as_index=False)
    .size()
    .rename(columns={"size": "tickets_received"})
    .sort_values("tickets_received", ascending=False)
)

fig = px.bar(
    tam_recv_df,
    x=tam_col, y="tickets_received", color="tickets_received",
    title="🎯 Who received the most tickets? Mind you....As a TAM",
    labels={"tickets_received": "Tickets", tam_col: "TAM"},
    template="simple_white"
)
fig.update_layout(showlegend=False)
fig.update_xaxes(categoryorder="total descending")
fig.show()



In [16]:
# 🔥 Which Closer closed the most tickets?  (excluding Status == "pending")
import plotly.express as px

closer_col = "Closer"
status_col = "Status"

# optional global excludes you may have defined earlier
EXCLUDE_CLOSERS = set([s.casefold() for s in globals().get("EXCLUDE_CLOSERS", [])])

src = df.copy()
# normalize status and drop pending rows
src[status_col] = src[status_col].astype("string").str.strip().str.lower()
src = src.loc[~src[status_col].eq("pending")]

closer_closed_df = (
    src
    .assign(**{closer_col: src[closer_col].astype("string").str.strip().str.replace(r"\s+", " ", regex=True)})
    .loc[lambda d:
         d[closer_col].notna()
         & (d[closer_col] != "")
         & (~d[closer_col].str.casefold().isin({"not found","nan","none"}))
         & (~d[closer_col].str.casefold().isin(EXCLUDE_CLOSERS))
    ]
    .groupby(closer_col, as_index=False)
    .size()
    .rename(columns={"size": "tickets_closed"})
    .sort_values("tickets_closed", ascending=False)
)

fig = px.bar(
    closer_closed_df,
    x=closer_col, y="tickets_closed", color="tickets_closed",
    title="🔥 Which Closer closed the most tickets? (excluding pending)",
    labels={"tickets_closed": "Tickets", closer_col: "Closer"},
    template="simple_white"
)
fig.update_layout(showlegend=False)
fig.update_xaxes(categoryorder="total descending")
fig.show()



In [17]:
# 4.4 Efficiency: average Time to Close (hours) — lower is better (by Closer, excluding 'pending')
import plotly.express as px

closer_col = "Closer"
ttc_col    = "Time to Close" if "Time to Close" in df.columns else ttc_col
status_col = "Status"

# use cleaned frame if you have it
src = df_clean.copy() if 'df_clean' in globals() else df.copy()

# normalize Status and remove 'pending'
src[status_col] = src[status_col].astype("string").str.strip().str.lower()
src = src.loc[~src[status_col].eq("pending")]

# optional global closer exclusions
EXCLUDE_CLOSERS = set([s.casefold() for s in globals().get("EXCLUDE_CLOSERS", {"bas","estefy","shub"})])

avg_ttc_df = (
    src
      .assign(**{closer_col: src[closer_col].astype("string").str.strip()})
      .loc[lambda d:
           d[closer_col].notna()
           & (d[closer_col] != "")
           & (~d[closer_col].str.casefold().isin({"not found","nan","none"}))
           & (~d[closer_col].str.casefold().isin(EXCLUDE_CLOSERS))
           & d[ttc_col].notna()]
      .groupby(closer_col, as_index=False)[ttc_col].mean()
      .rename(columns={ttc_col: "avg_hours"})
      .sort_values("avg_hours", ascending=True)
)

fig = px.bar(
    avg_ttc_df,
    x=closer_col, y="avg_hours", color="avg_hours",
    title="⚡ Fastest Closers (Avg Time to Close — excluding pending; lower is better)",
    labels={"avg_hours": "Avg hours", closer_col: "Closer"},
    template="simple_white"
)
fig.update_layout(showlegend=False)
fig.show()




In [15]:
# 🔄 Re-open Champion — latest week (by Closer)

import plotly.express as px
import pandas as pd

closer_col = "Closer"

# Find the reopen column in your CSV
reopen_col = None
for c in ["ReopenedAt", "reopenedat", "Re-opened At", "ReopenAt"]:
    if c in df.columns:
        reopen_col = c
        break

if reopen_col is None:
    print("No 'ReopenedAt' column found.")
else:
    # Parse datetime
    df[reopen_col] = pd.to_datetime(df[reopen_col], errors="coerce", dayfirst=True)

    # ISO week label from re-open timestamp
    iso_re = df[reopen_col].dt.isocalendar()
    df["re_week"] = iso_re.year.astype(str) + "-W" + iso_re.week.astype(str).str.zfill(2)

    # Latest week with actual re-opens
    valid_weeks = df.loc[df[reopen_col].notna(), "re_week"]
    if not len(valid_weeks):
        print("No re-open data available to compute weekly champions.")
    else:
        latest_week = valid_weeks.sort_values().iloc[-1]

        # Optional: exclude some closers globally
        EXCLUDE_CLOSERS = set([s.lower() for s in globals().get("EXCLUDE_CLOSERS", [])])

        reopens_latest = (
            df.loc[
                (df["re_week"] == latest_week)
                & df[reopen_col].notna()
                & df[closer_col].notna()
                & (df[closer_col].astype(str).str.strip() != "")
                & (~df[closer_col].astype(str).str.lower().isin({"not found","nan","none"}))
                & (~df[closer_col].astype(str).str.lower().isin(EXCLUDE_CLOSERS))
            ]
            .groupby(closer_col, as_index=False)
            .size()
            .rename(columns={"size": "tickets_reopened"})
            .sort_values("tickets_reopened", ascending=False)
        )

        fig = px.bar(
            reopens_latest,
            x="tickets_reopened",
            y=closer_col,
            orientation="h",
            color="tickets_reopened",
            title=f"🔄 Re-open Champion (Latest week in data: {latest_week})",
            labels={"tickets_reopened": "Tickets Reopened", closer_col: "Closer"},
            template="simple_white"
        )
        fig.update_layout(showlegend=False)
        fig.update_yaxes(categoryorder="total ascending")
        fig.show()


In [18]:
# --- Build Closer leaderboard (exclude pending/processing) ---
import numpy as np
import pandas as pd

closer_col  = "Closer"
client_col  = "ClientName"
status_col  = "Status"
ttc_col     = "Time to Close"   # hours
# find reopen col
reopen_col = next((c for c in ["ReopenedAt","reopenedat","Re-opened At","ReopenAt"] if c in df.columns), None)

# filter: remove pending / processing
work = df.copy()
work[status_col] = work[status_col].astype("string").str.strip().str.lower()
work = work.loc[~work[status_col].isin(["pending","processing"])]

# clean closer
work[closer_col] = work[closer_col].astype("string").str.strip()
work = work.loc[
    work[closer_col].notna()
    & (work[closer_col] != "")
    & (~work[closer_col].str.casefold().isin({"not found","nan","none"}))
]

# reopen flag
if reopen_col:
    work[reopen_col] = pd.to_datetime(work[reopen_col], errors="coerce", dayfirst=True)
    work["_reopened_flag"] = work[reopen_col].notna()
else:
    work["_reopened_flag"] = False

# aggregate per closer
agg = (
    work.groupby(closer_col)
        .agg(
            Tickets_Closed=("Status","size"),
            Unique_Clients=(client_col,"nunique"),
            Avg_Close_h=(ttc_col,"mean"),
            Median_Close_h=(ttc_col,"median"),
            Reopens=("_reopened_flag","sum")
        )
        .reset_index()
)

# derived metrics
agg["Closed per Client"] = agg["Tickets_Closed"] / agg["Unique_Clients"].replace(0, np.nan)
agg["Re-open Rate"]      = (agg["Reopens"] / agg["Tickets_Closed"].replace(0, np.nan)).fillna(0.0)

# scores (0–100) — lower avg hours is better
def _minmax(s):
    if s.nunique() <= 1: 
        return pd.Series(50.0, index=s.index)
    lo, hi = s.min(), s.max()
    return (s - lo) / (hi - lo) * 100

speed_score       = 100 - _minmax(agg["Avg_Close_h"])               # invert: lower hours → higher score
throughput_score  = _minmax(agg["Closed per Client"].fillna(0))
volume_score      = _minmax(agg["Tickets_Closed"])

agg["Speed Score (0–100)"]      = speed_score.round(1)
agg["Throughput Score (0–100)"] = throughput_score.round(1)
agg["Volume Score (0–100)"]     = volume_score.round(1)

# raw power (equal weights)
agg["POWER SCORE (0–100)"] = (
    0.33*agg["Speed Score (0–100)"] + 
    0.34*agg["Throughput Score (0–100)"] + 
    0.33*agg["Volume Score (0–100)"]
).round(1)

# re-open penalty (0..15) using percentile rank of reopen rate
def pct_rank(s):
    return s.rank(pct=True, method="average") if s.nunique() > 1 else pd.Series(0.5, index=s.index)
agg["Re-open Penalty (0–15)"] = (pct_rank(agg["Re-open Rate"]) * 15).round(1)

# adjusted power
agg["Adj POWER SCORE (0–100)"] = (agg["POWER SCORE (0–100)"] - agg["Re-open Penalty (0–15)"]).clip(lower=0)

# simple titles/tiers
def _title(r):
    return "Lightning Closer" if (r["Speed Score (0–100)"] >= 70 or r["Avg_Close_h"] <= 12) else "Steady Operator"

def _tier(score):
    if score >= 80: return "Mythic 🟣"
    if score >= 70: return "Legendary 🟡"
    if score >= 40: return "Epic 🔵"
    return "Rare 🟢"

agg["Title"] = agg.apply(_title, axis=1)
agg["Tier"]  = agg["Adj POWER SCORE (0–100)"].apply(_tier)

# final closer leaderboard (rename to match the card)
lb_pen_closer = (
    agg.rename(columns={
        closer_col: "Closer",
        "Tickets_Closed": "Tickets Closed",
        "Avg_Close_h": "Avg Close (h)",
        "Median_Close_h": "Median Close (h)"
    })
    .sort_values("Adj POWER SCORE (0–100)", ascending=False)
    .reset_index(drop=True)
)

lb_pen_closer.head()


,Closer,Tickets Closed,Unique_Clients,Avg Close (h),Median Close (h),Reopens,Closed per Client,Re-open Rate,Speed Score (0–100),Throughput Score (0–100),Volume Score (0–100),POWER SCORE (0–100),Re-open Penalty (0–15),Adj POWER SCORE (0–100),Title,Tier
0,Gui,29,2,12.460297,3.682222,1,14.500000,0.034483,88.5,100.0,100.0,96.2,10.0,86.2,Lightning Closer,Mythic 🟣
1,Zarmeen,12,1,7.585255,0.072222,1,12.000000,0.083333,93.0,82.8,39.3,71.8,12.9,58.9,Lightning Closer,Epic 🔵
2,Nelly,16,3,1.150747,0.498611,0,5.333333,0.000000,99.0,36.8,53.6,62.9,5.0,57.9,Lightning Closer,Epic 🔵
3,Annet,26,8,17.010481,5.206806,1,3.250000,0.038462,84.2,22.4,89.3,64.9,10.7,54.2,Lightning Closer,Epic 🔵
4,Estefy,15,4,0.893241,0.455556,0,3.750000,0.000000,99.2,25.9,50.0,58.0,5.0,53.0,Lightning Closer,Epic 🔵


In [25]:
# === Power Leaderboard (Closer) with Re-open Penalty + Normalized Scores ===
import numpy as np
import pandas as pd
from IPython.display import HTML, display

# ----------------------------
# 0) Inputs / helpers
# ----------------------------
closer_col  = "Closer"
client_col  = "ClientName"
status_col  = "Status"
ttc_col     = "Time to Close"  # hours
# find re-open column
reopen_col = next((c for c in ["ReopenedAt","reopenedat","Re-opened At","ReopenAt"] if c in df.columns), None)

def pct_rank01(s: pd.Series) -> pd.Series:
    """Percentile rank on [0,1]."""
    if s.size == 0:
        return s
    return s.rank(pct=True, method="average").astype(float)

def winsorize(s: pd.Series, lo=0.05, hi=0.95) -> pd.Series:
    """Clip to [lo, hi] quantiles to limit outliers."""
    if s.nunique() <= 1:
        return s
    a, b = s.quantile(lo), s.quantile(hi)
    return s.clip(a, b)

# ----------------------------
# 1) Filter & clean
# ----------------------------
src = (df_clean.copy() if 'df_clean' in globals() else df.copy())

# statuses
src[status_col] = src[status_col].astype("string").str.strip().str.lower()
src = src.loc[~src[status_col].isin(["pending","processing"])]

# closer text
src[closer_col] = src[closer_col].astype("string").str.strip()
src = src.loc[
    src[closer_col].notna()
    & (src[closer_col] != "")
    & (~src[closer_col].str.casefold().isin({"not found","nan","none"}))
]

# reopen flag
if reopen_col:
    src[reopen_col] = pd.to_datetime(src[reopen_col], errors="coerce", dayfirst=True)
    src["_reopened_flag"] = src[reopen_col].notna()
else:
    src["_reopened_flag"] = False

# Exclude Estefy (case-insensitive) before aggregations
# People to exclude (any capitalization)
EXCLUDE = {"estefy", "bas", "shub", "john doe"}  # add more names here

# Make sure the text is clean, then filter
src[closer_col] = src[closer_col].astype("string").str.strip()
src = src.loc[~src[closer_col].str.casefold().isin(EXCLUDE)]



# ----------------------------
# 2) Aggregate per closer
# ----------------------------
agg = (
    src.groupby(closer_col, as_index=False)
       .agg(
           Tickets_Closed=(status_col, "size"),
           Unique_Clients=(client_col, "nunique"),
           Avg_Close_h=(ttc_col, "mean"),
           Median_Close_h=(ttc_col, "median"),
           Reopens=("_reopened_flag", "sum")
       )
)


# Derived
agg["Closed per Client"] = agg["Tickets_Closed"] / agg["Unique_Clients"].replace(0, np.nan)
agg["Re-open Rate"]      = (agg["Reopens"] / agg["Tickets_Closed"].replace(0, np.nan)).fillna(0.0)

# ----------------------------
# 3) Normalized pillar scores (0..100)
# ----------------------------
# SPEED: lower Avg_Close_h is better → winsorize & invert via percentile
speed_base = winsorize(agg["Avg_Close_h"])
speed_score = (1 - pct_rank01(speed_base)) * 100  # invert
agg["Speed Score (0–100)"] = speed_score.round(1)

# THROUGHPUT (per-client) with empirical-Bayes shrinkage to global mean
pc = agg["Closed per Client"].fillna(0.0)
global_pc_mean = pc.replace([np.inf, -np.inf], np.nan).dropna().mean() if pc.notna().any() else 0.0
k = 10.0  # prior strength ~ “10 clients worth” of stabilization
n_clients = agg["Unique_Clients"].clip(lower=0).astype(float)
pc_shrunk = (pc * n_clients + global_pc_mean * k) / (n_clients + k)
thr_score = pct_rank01(winsorize(pc_shrunk)) * 100
agg["Throughput Score (0–100)"] = thr_score.round(1)

# VOLUME (tickets closed): log(1+x), winsorize, percentile
vol_log = np.log1p(agg["Tickets_Closed"].astype(float))
vol_score = pct_rank01(winsorize(vol_log)) * 100
agg["Volume Score (0–100)"] = vol_score.round(1)

# POWER (weights favor speed & per-client; cap to 0..100)
agg["POWER SCORE (0–100)"] = (
    0.35*agg["Speed Score (0–100)"] +
    0.35*agg["Throughput Score (0–100)"] +
    0.30*agg["Volume Score (0–100)"]
).clip(0, 100).round(1)

# ----------------------------
# 4) Re-open penalty & Adjusted Power
# ----------------------------
MAX_PENALTY = 15.0
def pct_rank(s):
    return s.rank(pct=True, method="average") if s.nunique() > 1 else pd.Series(0.5, index=s.index)

agg["Re-open Penalty (0–15)"] = (pct_rank(agg["Re-open Rate"]) * MAX_PENALTY).round(1)
agg["Adj POWER SCORE (0–100)"] = (agg["POWER SCORE (0–100)"] - agg["Re-open Penalty (0–15)"]).clip(lower=0)

# ----------------------------
# 5) Titles & Tiers (+ promotion)
# ----------------------------
def title_row(r):
    return "Lightning Closer" if (r["Speed Score (0–100)"] >= 70 or r["Avg_Close_h"] <= 12) else "Steady Operator"

def tier_row(score):
    if score >= 80:  return "Mythic 🟣"
    if score >= 70:  return "Legendary 🟡"
    if score >= 40:  return "Epic 🔵"
    return "Rare 🟢"

agg["Title"] = agg.apply(title_row, axis=1)
agg["Tier"]  = agg["Adj POWER SCORE (0–100)"].apply(tier_row)

# Promotion: Adj Power > 40 ⇒ tier +1
PROMOTE_THRESHOLD = 40.0
tier_order = ["Rare", "Epic", "Legendary", "Mythic"]
emoji_map  = {"Rare":"🟢","Epic":"🔵","Legendary":"🟡","Mythic":"🟣"}

def base_name(t):
    for name in tier_order:
        if name in str(t): return name
    return "Rare"

def bump_one(name):
    i = tier_order.index(name);  return tier_order[min(i+1, len(tier_order)-1)]

mask = agg["Adj POWER SCORE (0–100)"] > PROMOTE_THRESHOLD
agg.loc[mask, "Tier"] = (
    agg.loc[mask, "Tier"].map(base_name).map(bump_one).map(lambda n: f"{n} {emoji_map[n]}") + " ⬆︎"
)

# ----------------------------
# 6) Rank & medals
# ----------------------------
lb_pen = agg.rename(columns={
    closer_col: "Closer",
    "Avg_Close_h": "Avg Close (h)",
    "Median_Close_h": "Median Close (h)",
    "Tickets_Closed": "Tickets Closed"
}).sort_values("Adj POWER SCORE (0–100)", ascending=False).reset_index(drop=True)

lb_pen["Rank"] = range(1, len(lb_pen)+1)
lb_pen["🏆"]   = ["🥇" if i==0 else "🥈" if i==1 else "🥉" if i==2 else "" for i in range(len(lb_pen))]

# ----------------------------
# 7) Final view + styling
# ----------------------------
show = lb_pen[
    [
        "Rank","🏆","Closer","Title","Tier",
        "Adj POWER SCORE (0–100)","Re-open Penalty (0–15)",
        "Speed Score (0–100)","Throughput Score (0–100)","Volume Score (0–100)",
        "Avg Close (h)","Tickets Closed","Unique_Clients","Closed per Client",
        "Reopens","Re-open Rate"
    ]
].rename(columns={"Unique_Clients":"Unique Clients"})

def bg_tier(val):
    if isinstance(val, str) and "Mythic" in val:    return "background-color:#6f2dbd; color:white;"
    if isinstance(val, str) and "Legendary" in val: return "background-color:#f1c40f; color:black;"
    if isinstance(val, str) and "Epic" in val:      return "background-color:#3498db; color:white;"
    if isinstance(val, str) and "Rare" in val:      return "background-color:#27ae60; color:white;"
    return ""

def text_bold(val): return "font-weight:700;" if isinstance(val, str) and val else ""

display(HTML("""
<div style="background:#0f172a;color:#fff;padding:14px 16px;border-radius:12px;
            font-family:ui-sans-serif,-apple-system,Segoe UI,Roboto;margin:4px 0 10px;">
  <div style="font-weight:800;font-size:22px;">🏆 Power Leaderboard — Closer (normalized, w/ Re-open Penalty)</div>
  <div style="opacity:.95;font-size:14px;margin-top:4px">
    <b>Adjusted Power = 0.35·Speed + 0.35·Per-Client + 0.30·Volume − Re-open Penalty</b>.<br/>
    Normalization: percentiles (winsorized), log-volume, and empirical-Bayes per-client (fair to everyone).<br/>
    Promotion: score &gt; 40 ⇒ Tier +1 (Rare→Epic→Legendary→Mythic).<br/>
    Orange = Adjusted Power · <span style="color:#ef4444">Red</span> = Penalty ·
    Teal = Per-Client · Purple = Volume · Green→Red = Avg hours (lower is better).
  </div>
</div>
"""))

pmax   = float(show["Adj POWER SCORE (0–100)"].max())
vmax_t = float(show["Tickets Closed"].max())
vmax_pc= float(show["Closed per Client"].max())
penmax = float(show["Re-open Penalty (0–15)"].max()) or 15.0

styled = (
    show.style
      .set_table_styles([
          {"selector":"th","props":[("background","#e5e7eb"),("color","#111827"),
                                    ("font-weight","700"),("padding","8px 10px"),("border","0")]},
          {"selector":"td","props":[("padding","8px 10px"),("border","0")]},
          {"selector":"tbody tr:nth-child(odd)","props":[("background","#f9fafb")]},
          {"selector":"tbody tr:nth-child(even)","props":[("background","#ffffff")]},
          {"selector":"table","props":[("border-collapse","separate"),("border-spacing","0 4px")]}
      ])
      .hide(axis="index")
      .applymap(bg_tier, subset=["Tier"])
      .applymap(text_bold, subset=["Closer","Title"])
      .set_properties(subset=[
          "Adj POWER SCORE (0–100)","Re-open Penalty (0–15)",
          "Speed Score (0–100)","Throughput Score (0–100)","Volume Score (0–100)",
          "Avg Close (h)","Tickets Closed","Unique Clients","Closed per Client",
          "Reopens","Re-open Rate"
      ], **{"text-align":"right"})
      # Adjusted Power
      .background_gradient(cmap="YlOrRd", subset=["Adj POWER SCORE (0–100)"], vmin=0, vmax=pmax)
      .bar(subset=["Adj POWER SCORE (0–100)"], color="#ff7f50", vmin=0, vmax=pmax)
      # Penalty
      .bar(subset=["Re-open Penalty (0–15)"], color="#ef4444", vmin=0, vmax=penmax)
      # Component scores
      .background_gradient(cmap="PuBuGn", subset=["Speed Score (0–100)"], vmin=0, vmax=100)
      .background_gradient(cmap="BuGn",  subset=["Throughput Score (0–100)"], vmin=0, vmax=100)
      .background_gradient(cmap="Greys", subset=["Volume Score (0–100)"], vmin=0, vmax=100)
      # Raw pillars
      .background_gradient(cmap="RdYlGn_r", subset=["Avg Close (h)"])  # lower = greener
      .bar(subset=["Closed per Client"], color="#00b894", vmin=0, vmax=vmax_pc)
      .bar(subset=["Tickets Closed"],    color="#6c5ce7", vmin=0, vmax=vmax_t)
      .format({
          "Adj POWER SCORE (0–100)":"{:.1f}",
          "Re-open Penalty (0–15)":"{:.1f}",
          "Speed Score (0–100)":"{:.1f}",
          "Throughput Score (0–100)":"{:.1f}",
          "Volume Score (0–100)":"{:.1f}",
          "Avg Close (h)":"{:.2f}",
          "Closed per Client":"{:.2f}",
          "Re-open Rate":"{:.2%}"
      })
)

styled



/var/folders/zj/ms63hq9j0_q88w6nyhjmppmw0000gn/T/ipykernel_84864/4212878875.py:219: FutureWarning:

Styler.applymap has been deprecated. Use Styler.map instead.

/var/folders/zj/ms63hq9j0_q88w6nyhjmppmw0000gn/T/ipykernel_84864/4212878875.py:220: FutureWarning:

Styler.applymap has been deprecated. Use Styler.map instead.



Rank,🏆,Closer,Title,Tier,Adj POWER SCORE (0–100),Re-open Penalty (0–15),Speed Score (0–100),Throughput Score (0–100),Volume Score (0–100),Avg Close (h),Tickets Closed,Unique Clients,Closed per Client,Reopens,Re-open Rate
1,🥇,Nelly,Lightning Closer,Mythic 🟣 ⬆︎,81.2,4.6,94.4,88.9,72.2,1.15,16,3,5.33,0,0.00%
2,🥈,Gui,Steady Operator,Legendary 🟡 ⬆︎,69.4,9.2,38.9,100.0,100.0,12.46,29,2,14.50,1,3.45%
3,🥉,Zarmeen,Lightning Closer,Legendary 🟡 ⬆︎,60.0,12.5,55.6,94.4,66.7,7.59,12,1,12.00,1,8.33%
4,,Arslan,Lightning Closer,Legendary 🟡 ⬆︎,55.7,4.6,72.2,33.3,77.8,3.37,17,6,2.83,0,0.00%
5,,Mael,Steady Operator,Legendary 🟡 ⬆︎,55.0,10.8,33.3,83.3,83.3,13.87,18,4,4.50,1,5.56%
6,,Juani,Lightning Closer,Legendary 🟡 ⬆︎,54.3,4.6,77.8,66.7,27.8,3.30,2,1,2.00,0,0.00%
7,,Annet,Steady Operator,Legendary 🟡 ⬆︎,53.6,10.0,27.8,77.8,88.9,17.01,26,8,3.25,1,3.85%
8,,Tristan,Lightning Closer,Legendary 🟡 ⬆︎,45.9,13.3,61.1,55.6,61.1,6.86,8,3,2.67,1,12.50%
9,,Razvan,Lightning Closer,Legendary 🟡 ⬆︎,45.4,4.6,88.9,11.1,50.0,2.12,6,5,1.20,0,0.00%
10,,Marina,Lightning Closer,Legendary 🟡 ⬆︎,44.6,4.6,50.0,66.7,27.8,8.32,2,1,2.00,0,0.00%


In [32]:
# Display ONLY these 3 cards in this order: Nelly, Gui, Zarmeen
import numpy as np, pandas as pd
from IPython.display import HTML, display

names_in_order = ["Nelly", "Gui", "Zarmeen"]

# -- find a leaderboard dataframe
for _cand in ["lb_pen", "lb", "show"]:
    if _cand in globals() and isinstance(globals()[_cand], pd.DataFrame):
        df_lb = globals()[_cand].copy()
        break
else:
    raise RuntimeError("No leaderboard DF found (expected one of: lb_pen, lb, show).")

# -- resolve columns (works with your closer leaderboard)
def pick(df, options): 
    for o in options:
        if o in df.columns: return o

closer_c = pick(df_lb, ["Closer","TAM"])
title_c  = pick(df_lb, ["Title"])
tier_c   = pick(df_lb, ["Tier"])
adj_c    = pick(df_lb, ["Adj POWER SCORE (0–100)","POWER SCORE (0–100)","Adj POWER"])
pen_c    = pick(df_lb, ["Re-open Penalty (0–15)","Reopen Penalty (0–15)"])
rate_c   = pick(df_lb, ["Re-open Rate","Reopen Rate"])
avg_c    = pick(df_lb, ["Avg Close (h)","Average Close (h)","avg_hours"])
per_c    = pick(df_lb, ["Closed per Client"])
vol_c    = pick(df_lb, ["Tickets Closed","tickets_closed"])

need = [closer_c, title_c, tier_c, adj_c, avg_c, per_c, vol_c]
if any(x is None for x in need):
    raise RuntimeError("Leaderboard is missing required columns to render the cards.")

# -- pick the rows in the requested order
df_lb["_match"] = df_lb[closer_c].astype(str).str.strip().str.lower()
want = [n.lower() for n in names_in_order]
rows = []
for i, who in enumerate(want):
    hit = df_lb[df_lb["_match"] == who]
    if not len(hit):
        raise RuntimeError(f"'{names_in_order[i]}' not found in {closer_c} column.")
    rows.append(hit.iloc[0])

medals  = ["🥇","🥈","🥉"]
headers = [("#f59e0b","#b45309"),("#9ca3af","#6b7280"),("#b45309","#92400e")]  # gold/silver/bronze

cards = []
for i, r in enumerate(rows):
    name  = str(r[closer_c]); title = str(r.get(title_c,"—")); tier = str(r.get(tier_c,"—"))
    adj   = f'{float(r[adj_c]):.1f}'
    pen   = f'{float(r.get(pen_c,0) or 0):.1f}' if pen_c else "—"
    ratev = r.get(rate_c, np.nan); rate = "—" if pd.isna(ratev) else f'{float(ratev):.2%}'
    avg   = f'{float(r[avg_c]):.2f}'
    per   = f'{float(r[per_c]):.2f}'
    vol   = f'{float(r[vol_c]):.0f}'
    c1,c2 = headers[i]

    cards.append(f"""
      <div class="card">
        <div class="hdr" style="background:linear-gradient(90deg,{c1},{c2});">
          <span class="hdrtxt">{medals[i]} ACHIEVEMENT UNLOCKED</span>
        </div>
        <div class="body">
          <div class="big">CLOSER: {name}</div>
          <div class="meta">Title: “{title}”</div>
          <div class="meta">Tier: {tier}</div>
          <div class="power">Adjusted Power: {adj}</div>
          <div class="sub">(Penalty: {pen} · Re-open rate: {rate})</div>
          <div class="stats">
            <div>Avg Close (h): <b>{avg}</b></div>
            <div>Closed per Client: <b>{per}</b></div>
            <div>Tickets Closed: <b>{vol}</b></div>
          </div>
        </div>
      </div>
    """)

css = """
<style>
.cards{display:flex;gap:18px;flex-wrap:wrap}
.card{flex:1 1 31%;min-width:320px;border-radius:18px;background:#fff;
      box-shadow:0 10px 20px rgba(0,0,0,.08);overflow:hidden}
.hdr{padding:16px 18px;color:#111}
.hdrtxt{font-size:22px;font-weight:800;letter-spacing:.3px}
.body{padding:18px 22px;font-family:ui-sans-serif,-apple-system,Segoe UI,Roboto}
.big{font-size:40px;font-weight:800;margin:8px 0 6px}
.meta{font-size:18px;margin:2px 0}
.power{font-size:26px;font-weight:800;margin-top:10px}
.sub{opacity:.8;margin-top:2px}
.stats{font-size:18px;margin-top:16px;line-height:1.7}
</style>
"""
display(HTML(css + '<div class="cards">' + "".join(cards) + "</div>"))
